### OpenDART 문서 다운로드 및 텍스트 추출
  - 기능: 특정 공시 문서 다운로드 및 HTML 텍스트 추출
  - 대상: 지정된 접수번호(RCEPT_NO) 또는 기업별 최신 공시
  - 데이터소스: 금융감독원 전자공시시스템 (OpenDART)
  - 주요내용:
    - ZIP 파일 형태의 공시 문서 다운로드
    - HTML 파일에서 순수 텍스트 추출 (BeautifulSoup 사용)
    - 추출된 텍스트를 파일로 저장
    - 기업별 최신 공시 자동 선택 기능

In [ ]:
import os, io, zipfile
from pathlib import Path
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

DATA_DIR = Path('data')
DATA_DIR.mkdir(parents=True, exist_ok=True)

# API_KEY = os.environ.get('OPEN_DART_API_KEY')
API_KEY = '코드'
BASE = 'https://opendart.fss.or.kr/api'


In [5]:
# rcept_no가 없으면 회사명으로 최근 공시에서 선택
rcept_no = os.environ.get('DART_RCEPT_NO')
if not rcept_no and API_KEY:
    # corp map 조회
    import xmltodict
    url = f"{BASE}/corpCode.xml"
    params = {'crtfc_key': API_KEY}
    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    z = zipfile.ZipFile(io.BytesIO(r.content))
    name = [n for n in z.namelist() if n.lower().endswith('.xml')][0]
    xml_bytes = z.read(name)
    doc = xmltodict.parse(xml_bytes)
    lists = doc.get('result', {}).get('list', [])
    if isinstance(lists, dict):
        lists = [lists]
    df_map = pd.DataFrame([{'corp_name': it.get('corp_name'), 'corp_code': it.get('corp_code')} for it in lists])
    target_name = os.environ.get('DART_CORP_NAME', '삼성전자')
    row = df_map[df_map['corp_name'] == target_name].head(1)
    corp_code = None if row.empty else row.iloc[0]['corp_code']
    if corp_code:
        today = datetime.today().strftime('%Y%m%d')
        url_list = f"{BASE}/list.json"
        params = {'crtfc_key': API_KEY, 'corp_code': corp_code, 'bgn_de': '20240101', 'end_de': today, 'page_no': 1, 'page_count': 100}
        rl = requests.get(url_list, params=params, timeout=30)
        rl.raise_for_status()
        filings = pd.DataFrame(rl.json().get('list', []))
        rcept_no = None if filings.empty else str(filings.iloc[0]['rcept_no'])
else:
    filings = pd.DataFrame()


In [ ]:
filings.head() if 'filings' in globals() and not filings.empty else (rcept_no or 'No filings')

In [14]:
# 문서 ZIP 다운로드 및 추출
out_zip = DATA_DIR / f'dart_doc_{rcept_no}.zip' if rcept_no else None
out_dir = DATA_DIR / f'dart_doc_{rcept_no}' if rcept_no else None

if API_KEY and rcept_no:
    url_doc = f"{BASE}/document.xml"
    params = {'crtfc_key': API_KEY, 'rcept_no': rcept_no}
    rd = requests.get(url_doc, params=params, timeout=60)
    rd.raise_for_status()
    if out_zip:
        out_zip.write_bytes(rd.content)
        out_dir.mkdir(exist_ok=True)
        with zipfile.ZipFile(out_zip, 'r') as zf:
            zf.extractall(out_dir)
else:
    out_dir = None


In [ ]:
# HTML/XML 텍스트 추출 요약
if out_dir and out_dir.exists():
    texts = []
    
    # HTML 파일 처리
    for p in out_dir.rglob('*.htm*'):
        try:
            html = p.read_text(encoding='utf-8', errors='replace')
            soup = BeautifulSoup(html, 'lxml')
            texts.append(soup.get_text(separator=' ', strip=True))
        except Exception as e:
            print(f"HTML 파일 처리 오류: {p.name} - {e}")
            continue
    
    # XML 파일 처리 (OpenDART XBRL/XML 파일)
    for p in out_dir.rglob('*.xml'):
        try:
            # 다양한 인코딩으로 시도
            xml_content = None
            encodings = ['utf-8', 'euc-kr', 'cp949']
            for encoding in encodings:
                try:
                    with open(p, 'r', encoding=encoding) as f:
                        xml_content = f.read()
                    # 한글 텍스트 포함 확인
                    if '삼성전자' in xml_content or '보고서' in xml_content:
                        break
                except UnicodeDecodeError:
                    continue
                except Exception:
                    continue
            
            if xml_content:
                # XML 태그 제거하고 한글 텍스트 추출
                import re
                clean_text = re.sub(r'<[^>]+>', ' ', xml_content)
                clean_text = re.sub(r'\s+', ' ', clean_text).strip()
                
                if len(clean_text) > 100:  # 의미있는 텍스트만
                    texts.append(clean_text)
                    print(f"XML 파일 처리 성공: {p.name} ({encoding} 인코딩, {len(clean_text)}자)")
            else:
                print(f"XML 파일 인코딩 감지 실패: {p.name}")
                
        except Exception as e:
            print(f"XML 파일 처리 오류: {p.name} - {e}")
            continue
    
    if texts:
        combined_text = '\n\n'.join(texts)
        txt_path = out_dir / 'extracted_text.txt'
        txt_path.write_text(combined_text[:100000], encoding='utf-8', errors='replace')
        preview = combined_text[:1000]
        print(f"텍스트 추출 완료: {len(texts)}개 파일, 총 {len(combined_text)}자")
    else:
        preview = ''
        print("텍스트 추출 실패: 처리할 수 있는 파일이 없습니다.")
else:
    preview = ''
    print("문서 디렉터리가 존재하지 않습니다.")


In [ ]:
preview if preview else 'No document extracted'
